<a href="https://colab.research.google.com/github/sunshineluyao/ESG_TwoSources/blob/main/ESG_Abnormal_Return_MSCI_2020_1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Creat the Leader, Average, and Laggard Group

In [1]:
import pandas as pd
import numpy as np

In [38]:
df =pd.read_csv('https://raw.githubusercontent.com/sunshineluyao/ESG_TwoSources/main/Benchmark.csv',index_col='Unnamed: 0')
df.head()

,Symbol,Month,Size,Mom,Industry_Adjusted_BMR_Quarterly,Size_G,BMR_G,Mom_G,Benchmark,Monthly_Return,Portfoio_Return
0,A,1,26461.7,11.6,-0.128462,5,3.0,2.0,5.32,-3.2,-0.791667
1,AVB,1,29287.1,12.5,-0.210955,5,3.0,2.0,5.32,3.3,-0.791667
2,ECL,1,55628.3,16.3,-0.170082,5,3.0,2.0,5.32,1.6,-0.791667
3,ICE,1,51536.5,15.2,-0.171353,5,3.0,2.0,5.32,7.8,-0.791667
4,IQV,1,29980.8,16.7,-0.124864,5,3.0,2.0,5.32,0.5,-0.791667


In [39]:
df['Abnormal_Return']=df['Monthly_Return']-df['Portfoio_Return']
df.head()

,Symbol,Month,Size,Mom,Industry_Adjusted_BMR_Quarterly,Size_G,BMR_G,Mom_G,Benchmark,Monthly_Return,Portfoio_Return,Abnormal_Return
0,A,1,26461.7,11.6,-0.128462,5,3.0,2.0,5.32,-3.2,-0.791667,-2.408333
1,AVB,1,29287.1,12.5,-0.210955,5,3.0,2.0,5.32,3.3,-0.791667,4.091667
2,ECL,1,55628.3,16.3,-0.170082,5,3.0,2.0,5.32,1.6,-0.791667,2.391667
3,ICE,1,51536.5,15.2,-0.171353,5,3.0,2.0,5.32,7.8,-0.791667,8.591667
4,IQV,1,29980.8,16.7,-0.124864,5,3.0,2.0,5.32,0.5,-0.791667,1.291667


In [40]:
ESG = pd.read_csv('https://raw.githubusercontent.com/sunshineluyao/ESG_TwoSources/main/MSCI_ESG_Data_0102.csv')
ESG.columns=['Month','Symbol','ESG_Rating','ESG_Score']
ESG.head()

,Month,Symbol,ESG_Rating,ESG_Score
0,1,A,AAA,8.8
1,1,AAL,CCC,0.7
2,1,AAON,A,5.8
3,1,AAP,BBB,4.6
4,1,AAPL,A,6.4


In [41]:
ESG.isna().sum()

Month         0
Symbol        0
ESG_Rating    0
ESG_Score     0
dtype: int64

### Categorize by ESG_Score

In [42]:
Month_List = ESG['Month'].unique().tolist()
Month_List

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [43]:
df_ =pd.DataFrame()
for i in Month_List:
  df_temp = ESG[ESG.Month==i]
  df_temp['ESG_G']=pd.qcut(df_temp['ESG_Score'],5,labels=np.arange(1,6,1))
  df_ =df_.append(df_temp)
df_.tail()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

,Month,Symbol,ESG_Rating,ESG_Score,ESG_G
15560,11,ZBH,CCC,1.2,1
15561,11,ZBRA,A,6.1,4
15562,11,ZION,A,6.2,4
15563,11,ZTS,A,6.4,4
15564,11,ZUMZ,BBB,5.1,3


In [44]:
ESG =df_
ESG.head()

,Month,Symbol,ESG_Rating,ESG_Score,ESG_G
0,1,A,AAA,8.8,5
1,1,AAL,CCC,0.7,1
2,1,AAON,A,5.8,4
3,1,AAP,BBB,4.6,3
4,1,AAPL,A,6.4,4


In [45]:
df=df.merge(ESG, on=['Month','Symbol'])
df.head()

,Symbol,Month,Size,Mom,Industry_Adjusted_BMR_Quarterly,Size_G,BMR_G,Mom_G,Benchmark,Monthly_Return,Portfoio_Return,Abnormal_Return,ESG_Rating,ESG_Score,ESG_G
0,A,1,26461.7,11.6,-0.128462,5,3.0,2.0,5.32,-3.2,-0.791667,-2.408333,AAA,8.8,5
1,AVB,1,29287.1,12.5,-0.210955,5,3.0,2.0,5.32,3.3,-0.791667,4.091667,BBB,5.1,3
2,ECL,1,55628.3,16.3,-0.170082,5,3.0,2.0,5.32,1.6,-0.791667,2.391667,AAA,10.0,5
3,ICE,1,51536.5,15.2,-0.171353,5,3.0,2.0,5.32,7.8,-0.791667,8.591667,BBB,5.7,4
4,IQV,1,29980.8,16.7,-0.124864,5,3.0,2.0,5.32,0.5,-0.791667,1.291667,A,5.8,4


In [46]:
df['ESG_Group']=df['ESG_G'].apply(lambda x: 'Laggard' if x==1 else 'Leader' if x==5 else 'Average')
df.head()

,Symbol,Month,Size,Mom,Industry_Adjusted_BMR_Quarterly,Size_G,BMR_G,Mom_G,Benchmark,Monthly_Return,Portfoio_Return,Abnormal_Return,ESG_Rating,ESG_Score,ESG_G,ESG_Group
0,A,1,26461.7,11.6,-0.128462,5,3.0,2.0,5.32,-3.2,-0.791667,-2.408333,AAA,8.8,5,Leader
1,AVB,1,29287.1,12.5,-0.210955,5,3.0,2.0,5.32,3.3,-0.791667,4.091667,BBB,5.1,3,Average
2,ECL,1,55628.3,16.3,-0.170082,5,3.0,2.0,5.32,1.6,-0.791667,2.391667,AAA,10.0,5,Leader
3,ICE,1,51536.5,15.2,-0.171353,5,3.0,2.0,5.32,7.8,-0.791667,8.591667,BBB,5.7,4,Average
4,IQV,1,29980.8,16.7,-0.124864,5,3.0,2.0,5.32,0.5,-0.791667,1.291667,A,5.8,4,Average


# Step 2: Visualizing Abnormal Return Monthly

In [47]:
import plotly.express as px
fig = px.box(df, x="ESG_Group", y="Abnormal_Return",color='ESG_Group',animation_frame='Month',animation_group='Symbol')
fig.show()

In [48]:
ESG_summary = df.groupby(by=['Month','ESG_Group'],as_index=False)['Abnormal_Return'].agg(['mean','std','count']).reset_index()
ESG_summary['sqrt_count']=ESG_summary['count'].apply(lambda x: np.sqrt(x))
ESG_summary['mean_std']=ESG_summary['std']/ESG_summary['sqrt_count']
ESG_summary.head()

,Month,ESG_Group,mean,std,count,sqrt_count,mean_std
0,1,Average,-0.204059,8.180662,863,29.376862,0.278473
1,1,Laggard,0.442630,9.463820,289,17.000000,0.556695
2,1,Leader,1.121462,7.246635,263,16.217275,0.446847
3,2,Average,-0.215706,8.372980,863,29.376862,0.285020
4,2,Laggard,1.491456,11.936147,289,17.000000,0.702126


In [49]:
import plotly.express as px
fig = px.line(ESG_summary, x="Month", y="mean", color="ESG_Group", error_y="mean_std")
fig.show()

In [50]:
ESG_summary = ESG_summary[ESG_summary['ESG_Group'].isin(['Leader', 'Laggard'])]

In [51]:
import plotly.express as px
fig = px.line(ESG_summary, x="Month", y="mean", color="ESG_Group", error_y="mean_std")
fig.show()

# Step 5: Conduct Two-group t-test for each month

In [52]:
pip install pingouin

In [53]:
import pingouin as pg


Two-Sample T-Test

https://pingouin-stats.org/generated/pingouin.ttest.html#pingouin.ttest

### Test Monthly the Leaders' Abornormal Return >0?

In [58]:
Leader = df[(df['Month']==1) & (df['ESG_Group']=='Leader')]['Abnormal_Return']
t_test = pg.ttest(Leader, 0)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,2.509725,262,two-sided,0.012686,"[0.24, 2.0]",0.154756,1.501,0.705596


In [59]:
df_ = pd.DataFrame()
Month_List =df.Month.unique().tolist()
for i in Month_List:
  Leader =  df[(df['Month']==i) & (df['ESG_Group']=='Leader')]['Abnormal_Return']
  t_test = pg.ttest(Leader, 0)
  t_test['Month']=i
  df_=df_.append(t_test)

In [60]:
df_

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power,Month
T-test,2.509725,262,two-sided,0.012686,"[0.24, 2.0]",0.154756,1.501,0.705596,1
T-test,-0.258230,262,two-sided,0.796432,"[-1.0, 0.77]",0.015923,0.071,0.057617,2
T-test,1.508329,262,two-sided,0.132675,"[-0.47, 3.56]",0.093008,0.212,0.324044,3
T-test,-1.269640,262,two-sided,0.205339,"[-3.12, 0.67]",0.078289,0.153,0.244165,4
T-test,0.139721,262,two-sided,0.888988,"[-1.21, 1.39]",0.008616,0.07,0.052223,5
T-test,1.007838,262,two-sided,0.314462,"[-0.47, 1.47]",0.062146,0.114,0.171100,6
T-test,1.897563,262,two-sided,0.058853,"[-0.05, 2.74]",0.117009,0.405,0.472411,7
T-test,1.210042,262,two-sided,0.227353,"[-0.45, 1.9]",0.074614,0.142,0.226092,8
T-test,0.649735,262,two-sided,0.516433,"[-0.67, 1.34]",0.040064,0.085,0.099220,9
T-test,1.329981,262,two-sided,0.184681,"[-0.36, 1.84]",0.082010,0.165,0.263271,10


In [61]:
df_2 = pd.DataFrame()
Month_List =df.Month.unique().tolist()
for i in Month_List:
  Leader = df[(df['Month']==i) & (df['ESG_Group']=='Leader')]['Abnormal_Return']
  Laggard = df[(df['Month']==i) & (df['ESG_Group']=='Laggard')]['Abnormal_Return']
  t_test = pg.ttest(Leader, Laggard,paired=False)
  t_test['Month']=i
  df_2=df_2.append(t_test)

In [62]:
df_2

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power,Month
T-test,0.950945,534.684344,two-sided,0.342062,"[-0.72, 2.08]",0.080049,0.147,0.155202,1
T-test,-1.927577,483.624501,two-sided,0.054493,"[-3.25, 0.03]",0.160775,0.577,0.469503,2
T-test,0.924248,548.731295,two-sided,0.355764,"[-1.5, 4.18]",0.078593,0.144,0.151307,3
T-test,-0.625961,539.395695,two-sided,0.531606,"[-3.92, 2.02]",0.052763,0.115,0.094781,4
T-test,0.314840,542.642122,two-sided,0.753004,"[-1.49, 2.05]",0.026859,0.1,0.061415,5
T-test,1.298513,549.055022,two-sided,0.194656,"[-0.49, 2.39]",0.109956,0.215,0.251384,6
T-test,2.688456,528.308735,two-sided,0.007405,"[0.68, 4.34]",0.230290,3.149,0.769626,7
T-test,2.513743,539.500461,two-sided,0.012236,"[0.44, 3.62]",0.214679,2.032,0.710468,8
T-test,1.430712,529.843145,two-sided,0.153102,"[-0.36, 2.29]",0.122509,0.257,0.300164,9
T-test,2.340982,546.027499,two-sided,0.019592,"[0.32, 3.65]",0.197826,1.355,0.639588,10


# Overall test

In [65]:
Leader =df[df['ESG_Group']=='Leader']['Abnormal_Return']
Laggard=df[df['ESG_Group']=='Laggard']['Abnormal_Return']

In [66]:
t_test = pg.ttest(Leader, 0)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,2.243726,2892,two-sided,0.024925,"[0.06, 0.86]",0.041715,0.259,0.611431


In [67]:
t_test = pg.ttest(Leader, Laggard, paired=False)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,3.042537,6066.90695,two-sided,0.002356,"[0.32, 1.5]",0.07775,2.921,0.856666


In [68]:
df.to_csv("ESG_MSCI_FINAL.csv")